In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Radial-symmetric reaction–diffusion + growth model
@author: Kelly Teitel
"""

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import pandas as pd
import os
from datetime import datetime

# ===========================================
# ------------ PARAMETERS -------------------
# ===========================================

GRID_SIZE = 400
DT = 0.05
MAX_FRAMES = 100

# Diffusion constants
D_PRODUCT = 5.0
D_SUBSTRATE = 5.0

# Radial geometry
RED_RADIUS = 30      # inner core
GREEN_RADIUS = 60    # outer shell (must be larger)

# Initial concentrations
SUBSTRATE_INIT = 100.0
PRODUCT_INIT = 0.0

# Enzyme kinetics
KM = 0.05
ENZYME_INIT = 1.0
MAX_ENZYME_ACTIVITY = 0.1

# Growth kinetics
GROWTH_RATE_PER_HOUR = np.log(2)
GROWTH_KM = 0.1

# ===========================================
# ------------ RADIAL GRID ------------------
# ===========================================

center = GRID_SIZE // 2
x = np.arange(GRID_SIZE)
y = np.arange(GRID_SIZE)
X, Y = np.meshgrid(x, y, indexing="ij")
R = np.sqrt((X - center)**2 + (Y - center)**2)

# ===========================================
# ------------ HELPER FUNCTIONS -------------
# ===========================================

def radial_average(field, R, dr=1.0):
    r_max = int(R.max())
    bins = np.arange(0, r_max + dr, dr)
    means = np.zeros(len(bins) - 1)

    for k in range(len(bins) - 1):
        mask = (R >= bins[k]) & (R < bins[k + 1])
        means[k] = field[mask].mean() if np.any(mask) else np.nan

    return 0.5 * (bins[:-1] + bins[1:]), means



def diffusion_2d(field, D, dt):
    lap = (
        np.roll(field, 1, axis=0) + np.roll(field, -1, axis=0) +
        np.roll(field, 1, axis=1) + np.roll(field, -1, axis=1) -
        4 * field
    )
    return field + D * dt * lap


# ===========================================
# ------------ INITIALIZATION ----------------
# ===========================================

timestamp = datetime.now().strftime("run_%Y-%m-%d_%H-%M-%S")
output_dir = os.path.join("simulation_outputs", timestamp)
os.makedirs(output_dir, exist_ok=True)

# Fields
grid = np.zeros((GRID_SIZE, GRID_SIZE), dtype=int)
substrate = np.zeros((GRID_SIZE, GRID_SIZE), dtype=float)
product = np.full((GRID_SIZE, GRID_SIZE), PRODUCT_INIT, dtype=float)
enzyme = np.zeros((GRID_SIZE, GRID_SIZE), dtype=float)

# --- CELL LAYOUT (RED INSIDE, GREEN OUTSIDE) ---
red_cells = R <= RED_RADIUS
green_cells = (R > RED_RADIUS) & (R <= GREEN_RADIUS)

grid[red_cells] = 1
grid[green_cells] = 2
enzyme[green_cells] = ENZYME_INIT

# Substrate starts outside aggregate
substrate[R > GREEN_RADIUS] = SUBSTRATE_INIT

# ===========================================
# ------------ VISUALIZATION ----------------
# ===========================================

fig, axes = plt.subplots(1, 2, figsize=(10, 5))

axes[0].set_title("Substrate")
axes[1].set_title("Product")

im_sub = axes[0].imshow(substrate, cmap="Blues", vmin=0, vmax=SUBSTRATE_INIT)
im_prod = axes[1].imshow(product, cmap="Reds", vmin=0, vmax=1.0)

for ax in axes:
    ax.axis("off")

frames = []

# ===========================================
# ------------ SIMULATION LOOP ---------------
# ===========================================

for frame in range(MAX_FRAMES):

    # Enzymatic conversion
    vmax = enzyme * MAX_ENZYME_ACTIVITY
    rate = vmax * substrate / (KM + substrate)
    substrate -= rate * DT
    product += rate * DT
    substrate = np.clip(substrate, 0, None)

    # Diffusion
    substrate = diffusion_2d(substrate, D_SUBSTRATE, DT)
    product   = diffusion_2d(product, D_PRODUCT, DT)


    # No-flux at r = 0
    product[center, center] = product[center + 1, center]

    # Numerical safety
    substrate = np.clip(substrate, 0, None)

    # Cell growth
    new_grid = grid.copy()
    new_enzyme = enzyme.copy()

    for cell_type in [1, 2]:
        positions = np.argwhere(grid == cell_type)
        np.random.shuffle(positions)

        for i, j in positions:
            p = product[i, j]
            grow_prob = GROWTH_RATE_PER_HOUR * DT * p / (GROWTH_KM + p)

            if np.random.rand() < grow_prob:
                nbrs = [(i + di, j + dj)
                        for di in [-1, 0, 1]
                        for dj in [-1, 0, 1]
                        if not (di == 0 and dj == 0)]

                nbrs = [(ni % GRID_SIZE, nj % GRID_SIZE) for ni, nj in nbrs]
                empty = [n for n in nbrs if grid[n] == 0]

                if empty:
                    ni, nj = empty[np.random.randint(len(empty))]
                    new_grid[ni, nj] = cell_type

                    if cell_type == 2:
                        parent = new_enzyme[i, j]
                        parent = min(parent + 0.01, 2.0)
                        new_enzyme[i, j] = parent
                        new_enzyme[ni, nj] = parent

    grid = new_grid
    enzyme = new_enzyme

    # Save radial profile
    r_vals, prod_r = radial_average(product, R)
    pd.DataFrame({"r": r_vals, "product": prod_r}).to_csv(
        f"{output_dir}/radial_product_{frame:04d}.csv", index=False
    )

    # Update animation frames
    frames.append([
        axes[0].imshow(substrate, cmap="Blues", vmin=0, vmax=SUBSTRATE_INIT, animated=True),
        axes[1].imshow(product, cmap="Reds", vmin=0, vmax=max(1e-6, product.max()), animated=True)
    ])

    print(f"Frame {frame:04d} | Cell density = {np.mean(grid > 0):.3f}")

# ===========================================
# ------------ SAVE VIDEO -------------------
# ===========================================

ani = FuncAnimation(fig, lambda i: frames[i], frames=len(frames), blit=True)
ani.save(
    os.path.join(output_dir, "substrate_to_product.gif"),
    writer="pillow",
    fps=10
)
plt.close()


Frame 0000 | Cell density = 0.071
Frame 0001 | Cell density = 0.071
Frame 0002 | Cell density = 0.071
Frame 0003 | Cell density = 0.071
Frame 0004 | Cell density = 0.071
Frame 0005 | Cell density = 0.071
Frame 0006 | Cell density = 0.071
Frame 0007 | Cell density = 0.071
Frame 0008 | Cell density = 0.071
Frame 0009 | Cell density = 0.071
Frame 0010 | Cell density = 0.071
Frame 0011 | Cell density = 0.071
Frame 0012 | Cell density = 0.071
Frame 0013 | Cell density = 0.071
Frame 0014 | Cell density = 0.071
Frame 0015 | Cell density = 0.071
Frame 0016 | Cell density = 0.071
Frame 0017 | Cell density = 0.071
Frame 0018 | Cell density = 0.071
Frame 0019 | Cell density = 0.071
Frame 0020 | Cell density = 0.071
Frame 0021 | Cell density = 0.071
Frame 0022 | Cell density = 0.071
Frame 0023 | Cell density = 0.071
Frame 0024 | Cell density = 0.071
Frame 0025 | Cell density = 0.071
Frame 0026 | Cell density = 0.071
Frame 0027 | Cell density = 0.071
Frame 0028 | Cell density = 0.071
Frame 0029 | C